In [4]:
import open3d as o3d

dataset = o3d.data.EaglePointCloud()
pcd = o3d.io.read_point_cloud(dataset.path)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: EGL: Failed to get EGL display: Success
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.



(python:2198): Gtk-WARNING **: 19:23:35.081: gtk_disable_setlocale() must be called before gtk_init()
